In [2]:
import pandas
import numpy as np
import tensorflow as tf
from keras.utils import np_utils
# CNN related
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import GRU
#from keras.layers.recurrent import LSTM, GRU
from keras.layers import Conv1D, MaxPooling1D, AtrousConvolution1D, RepeatVector
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers.wrappers import Bidirectional
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.initializers import *
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE # doctest: +NORMALIZE_WHITESPACE
from imblearn.over_sampling import RandomOverSampler # doctest: +NORMALIZE_WHITESPACE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import KMeansSMOTE
from imblearn.over_sampling import BorderlineSMOTE # doctest: +NORMALIZE_WHITESPACE
from imblearn.combine import SMOTETomek # doctest: +NORMALIZE_WHITESPACE

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from keras import backend as K


##################
# AUC for a binary classifier
def auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
    
##################
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    #print(FP/N)
    return FP/N

##################
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P

class ecgcnn: 
    np.set_printoptions(threshold=np.inf)
    def __init__(self, base_path, stkid, infosize, onehotsize ):
        self.stkid = stkid
        fname = base_path + stkid + '.csv'
        self.INFOSIZE = infosize
        self.ONEHOTSIZE = onehotsize
        self.load_data(fname)
        self.EMB_SIZE = self.INFOSIZE+self.ONEHOTSIZE
        #self.ONEHOTSIZE = onehotsize
        

##################
    def load_data(self, fname):
        # load csv data
        mat = pandas.read_csv(fname, sep=",", header=0, error_bad_lines=False).as_matrix()

        self.info = []
        self.sid=[]
        for i in range(self.INFOSIZE):
            self.info.append(self.matrix_col(mat, i))   
        self.sid.append(self.matrix_col(mat,16))
       # print(self.sid) 
        #x = self.find_category(self.sid)
        #print(x)
##################        
    '''
    def find_category0(self, sid):
        print(sid[0][0])
        y = list(map(int, sid[3:-1])) 
        #y = map(eval,sid[3:-1])
        print(y)    
        for n in range(len(sid)):
            sidnum = [0]*51 
            if ( y == 11 ):
                print("456") 
                sidnum [n]=sidnum [n]+1
                print(sidnum)
                return  sidnum
    '''
    def find_category(self, sid):
        result = []
        for sidstr in sid[0]:
            index = int(sidstr.replace('sid', ''))
            result.append(index)
            
        return result
        
    '''           
            sidnum = [0]*51
            #print(index)
            sidnum[index-1] = 1
            #print(sidnum)
            result.append(sidnum)
           
            return result
    ''' 
######################################################
    def matrix_col(self, matrix, i):
        return [row[i] for row in matrix]    
    
######################################################
    def normalize(self, data):
        #return (np.array(data)-np.mean(data))/np.std(data) 
        std = np.std(data)
        if (std == 0):
            data2 = np.zeros(len(data))
            return data2
        return (np.array(data)-np.mean(data))/std
    
######################################################
    
    def backtest_training_data(self,num,s):
        x_train0, y_train_onehot = [], []  
        x_train1, x_test1, y_train1, y_test1 = [], [], [], []
        # normalize
        nvals = []
        for cval in self.info:
            nvals.append(self.normalize(cval))
        
        # training preparation            
        l2t = nvals
        x_train0 = np.column_stack(l2t)
        y_train_onehot = self.find_category(self.sid)

        x_train = []
        for rec in x_train0:
            x_train.append(rec)
        
        x_train = np.array(x_train)
        #y_train_onehot = np_utils.to_categorical(y_train)
        y_train_onehot = np.array(y_train_onehot)

        if num==0:
            n = 40
        if num==1:
            n = 38
        if num==2:
            n = 16
        if num==3:
            n = 81
        x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train_onehot, test_size=0.2, random_state=None)
        
        
        if s==0:
            x_res, y_res = x_train1, y_train1
        else:    
            if s==1:
                sm = SMOTE(random_state=0)
            if s==2:
                sm = SVMSMOTE(random_state=0)
            if s==3:
                sm = RandomOverSampler(random_state=0)
            if s==4:
                sm = BorderlineSMOTE(random_state=0) 
            if s==5:
                sm = SMOTETomek(random_state=0)
            #sm = KMeansSMOTE(random_state=0)  #X need 25 samples
            
            x_res, y_res = sm.fit_resample(x_train1, y_train1)
            
            #print(x_train)
            #print('Resampled dataset shape %s' % Counter(y_res))

        x_train_new = []
        for rec in x_res:
            x_train_new.append([rec])
            #x_train_new.append(np.reshape(rec,(-1,self.EMB_SIZE)))
        x_test_new = []
        for rec1 in x_test1:
            x_test_new.append([rec1])
                
        x_train_new = np.array(x_train_new)  
        x_test_new = np.array(x_test_new)
        #print(x_train_new)
        
        y_train_onehot_new = []
        y_test_new = [] 
        y_test_new1 = [] 
        
        for sid in y_res:
            index = int(sid)
            sidnum = [0]*n
           
            #print(index)
            sidnum[index-1] = 1
            #print(index)
            #print(sidnum)
            y_train_onehot_new.append(sidnum)  
        for sidstr in y_test1:
            index = int(sidstr)
            sidnum1 = [0]*n
            
            #print(index)
            sidnum1[index-1] = 1
            #print(sidnum)
            y_test_new.append(sidnum1) 
            y_test_new1.append([sidnum1])

        return x_train_new, y_train_onehot_new, x_test_new, y_test_new, y_test_new1

######################################################
    def build_model(self,num):
        return self.simple_model(num)

######################################################
    def simple_model(self,num):
        model = Sequential()
        model.add(Conv1D(input_shape=(1, self.EMB_SIZE), filters=128, kernel_size=16, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.5))

        model.add(Conv1D(filters=64, kernel_size=8, padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.5))
       
        model.add(Flatten())
        model.add(Dense(128))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        if num==0:
            model.add(Dense(40))
        if num==1:
            model.add(Dense(38))
        if num==2:
            model.add(Dense(16))
        if num==3:
            model.add(Dense(81))
            
        model.add(Activation('softmax'))

        opt = Nadam(lr=0.002)
        
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[auc])
        #model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        #print(model.summary())
        return model

######################################################
    def train_model(self, model, x_train, y_train_onehot, x_test1, y_test1 ):
        model_name = 'cnn.'+self.stkid+'.model'
        input_shape = x_train[0].shape
        #reduce_lr = ReduceLROnPlateau(monitor='roc_auc_val', factor=0.8, patience=30, min_lr=0.000001, verbose=2)
        reduce_lr = ReduceLROnPlateau(monitor='roc_accuracy', factor=0.8, patience=30, min_lr=0.000001, verbose=2)
        checkpointer = ModelCheckpoint(filepath=model_name, verbose=0, save_best_only=True)
        
        train_history = model.fit(np.array(x_train),np.array(y_train_onehot), epochs = 200, batch_size = 10, verbose=2, \
                                  validation_data=(np.array(x_test1), np.array(y_test1)), callbacks=[reduce_lr, checkpointer], shuffle=True)
        '''
        train_history = model.fit(np.array(x_train),np.array(y_train_onehot), epochs = 200, batch_size = 100, verbose=0, \
                                   callbacks=[reduce_lr, checkpointer], shuffle=True)
        '''
        #show_train_history(train_history, 'accuracy', 'val_accuracy')
        #show_train_history(train_history, 'auc', 'val_auc')
        
        #show_train_history(train_history, 'loss', 'val_loss')
        return train_history

######################################################
    def do_backtest(self,num,s):

        # cut data 
        x_train, y_train_onehot, x_test1, y_test1, y_test2 = self.backtest_training_data(num,s)
        # build model 
        model = self.build_model(num)
        train_history = self.train_model(model, x_train, y_train_onehot, x_test1, y_test1 )
        
        if num==0:
            n = 40
        if num==1:
            n = 38
        if num==2:
            n = 16
        if num==3:
            n = 81
            
        count = 0
        total = 0
        #print(x_test1)
        #print(np.array(y_test2))
        #print("x : "+str(x_test1.shape)+" / y : "+str(np.array(y_test2).shape))
        for x_test, y_test in zip(x_test1, np.array(y_test2)):
            
            prediction = model.predict_classes(np.array([x_test]),verbose=0)
            #print(proba[0], proba[1])
            #print(datestr, prediction[0], y_test_onehot[0], scores[1], last_close)
            
      
            index = int(prediction)+1
            #print("prediction: "+str(index))
        
            sidnum = [0]*n
            #print(index)
            sidnum[index-1] = 1
            sidnum_x = np.array(sidnum)
            
            total += 1
            
            if (y_test[0][index-1] == sidnum_x[index-1]):
                count += 1
        if (total == 0):
            return 0, 0
        
    
        #print(total)
        #print(count)
        return count, total
        
######################################################
"""
def show_train_history(train_history, train, validation):
    plt.plot(train_history.history[train])
    #plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
"""
def main():
    print("start")
    stkid = 'ALL_BMI_2' 
    TID = 'BMI'
    DA = ["AUSAIC Data","MIT-BIH Arrhythmia Data","MIT-BIH Normal Sinus Rhythm Data","QT Data"]
    SMO = ["Original","SMOTE","SVMSMOTE","ROS","BorderlineSMOTE","SMOTETomek"]
    ADD = ["/home/keg/桌面/ecg_data/AUSAIC Data/BMI/",
          "/home/keg/桌面/ecg_data/MIT-BIH Arrhythmia Data/BMI/",
          "/home/keg/桌面/ecg_data/MIT-BIH Normal Sinus Rhythm Data/BMI/",
          "/home/keg/桌面/ecg_data/QT Data/BMI/"]
    NUMB = [0, 1, 2]
    NUMBE = [0, 1, 2, 3, 4, 5]
    info_size = 15
    onehot_size = 0
    #file = open('/home/keg/桌面/ecg_data/BMI_accuracy-1.csv', 'w')
    #file.write('Method,DATA,Ture/Total,accuracy'+'\n')
    for num in NUMB:
        base_path = ADD[num]
        #S=0  #SMOTE /S=1  #SVMSMOTE /S=2  #ROS  /S=3  #BorderlineSMOTE  /S=4  #SMOTETomek
        for S in NUMBE:
            ecg = ecgcnn(base_path, stkid, info_size, onehot_size )
            count, total = ecg.do_backtest(num,S)
            hitrate = 0.0
            if (total > 0):
                hitrate = float(count)/total
                #print(SMO[S]+","+DA[num]+",("+str(count)+"/"+str(total)+"),"+str(hitrate))
                print(SMO[S]+","+DA[num])
                plan =SMO[S]+","+DA[num]+",("+str(count)+"/"+str(total)+"),"+str(hitrate)+'\n'
                #file = open('/home/keg/桌面/ecg_data/BMI_accuracy.txt', 'w')
                #file.write(plan)
    #file.close()
    print("finish")
    #xt, yt = ecg.backtest_training_data()
    #print(xt)
    #print(yt[-1])
    #for stkid in stklist:    
    #    model = ecgcnn(base_path, stkid,info_size)
                      
if (__name__ == "__main__"):
    main()


start


/home/keg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:80: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
W0118 16:00:37.177772 139967354611456 deprecation_wrapper.py:119] From /home/keg/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 3476 samples, validate on 869 samples
Epoch 1/200
 - 45s - loss: 2.7897 - auc: 0.8359 - val_loss: 2.2024 - val_auc: 0.9478


/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `roc_accuracy` which is not available. Available metrics are: val_loss,val_auc,loss,auc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/200
 - 18s - loss: 2.2904 - auc: 0.9111 - val_loss: 1.7593 - val_auc: 0.9604
Epoch 3/200
 - 18s - loss: 2.1934 - auc: 0.9197 - val_loss: 1.6626 - val_auc: 0.9661
Epoch 4/200
 - 18s - loss: 2.0991 - auc: 0.9261 - val_loss: 1.6838 - val_auc: 0.9661
Epoch 5/200
 - 18s - loss: 2.0962 - auc: 0.9265 - val_loss: 1.5990 - val_auc: 0.9702
Epoch 6/200
 - 18s - loss: 1.9884 - auc: 0.9348 - val_loss: 1.5954 - val_auc: 0.9682
Epoch 7/200
 - 18s - loss: 1.9893 - auc: 0.9339 - val_loss: 1.5624 - val_auc: 0.9703
Epoch 8/200
 - 18s - loss: 1.9590 - auc: 0.9361 - val_loss: 1.5490 - val_auc: 0.9714
Epoch 9/200
 - 18s - loss: 1.9395 - auc: 0.9369 - val_loss: 1.5476 - val_auc: 0.9708
Epoch 10/200
 - 18s - loss: 1.9220 - auc: 0.9390 - val_loss: 1.5003 - val_auc: 0.9737
Epoch 11/200
 - 18s - loss: 1.8998 - auc: 0.9388 - val_loss: 1.5749 - val_auc: 0.9705
Epoch 12/200
 - 18s - loss: 1.9080 - auc: 0.9382 - val_loss: 1.5068 - val_auc: 0.9729
Epoch 13/200
 - 18s - loss: 1.8942 - auc: 0.9411 - val_loss: 1

...

Train on 1966 samples, validate on 261 samples
Epoch 1/200
 - 51s - loss: 1.3616 - auc: 0.9203 - val_loss: 1.4538 - val_auc: 0.9719


/home/keg/anaconda3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `roc_accuracy` which is not available. Available metrics are: val_loss,val_auc,loss,auc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/200
 - 9s - loss: 0.7232 - auc: 0.9814 - val_loss: 0.4015 - val_auc: 0.9969
Epoch 3/200
 - 9s - loss: 0.6246 - auc: 0.9835 - val_loss: 0.2971 - val_auc: 0.9970
Epoch 4/200
 - 9s - loss: 0.5348 - auc: 0.9881 - val_loss: 0.2511 - val_auc: 0.9977
Epoch 5/200
 - 9s - loss: 0.4714 - auc: 0.9902 - val_loss: 0.2222 - val_auc: 0.9984
Epoch 6/200
 - 9s - loss: 0.4671 - auc: 0.9892 - val_loss: 0.2168 - val_auc: 0.9974
Epoch 7/200
 - 9s - loss: 0.4449 - auc: 0.9911 - val_loss: 0.2081 - val_auc: 0.9980
Epoch 8/200
 - 9s - loss: 0.3996 - auc: 0.9923 - val_loss: 0.2168 - val_auc: 0.9971
Epoch 9/200
 - 9s - loss: 0.3924 - auc: 0.9920 - val_loss: 0.1918 - val_auc: 0.9978
Epoch 10/200
 - 9s - loss: 0.3905 - auc: 0.9905 - val_loss: 0.1621 - val_auc: 0.9988
Epoch 11/200
 - 9s - loss: 0.3658 - auc: 0.9931 - val_loss: 0.1698 - val_auc: 0.9983
Epoch 12/200
 - 9s - loss: 0.3358 - auc: 0.9946 - val_loss: 0.1765 - val_auc: 0.9985
Epoch 13/200
 - 9s - loss: 0.3623 - auc: 0.9933 - val_loss: 0.1620 - val_

Epoch 99/200
 - 9s - loss: 0.1828 - auc: 0.9971 - val_loss: 0.0795 - val_auc: 0.9997
Epoch 100/200
 - 9s - loss: 0.1728 - auc: 0.9974 - val_loss: 0.1157 - val_auc: 0.9996
Epoch 101/200
 - 9s - loss: 0.1721 - auc: 0.9980 - val_loss: 0.0842 - val_auc: 0.9997
Epoch 102/200
 - 9s - loss: 0.2099 - auc: 0.9944 - val_loss: 0.0835 - val_auc: 0.9997
Epoch 103/200
 - 9s - loss: 0.2075 - auc: 0.9968 - val_loss: 0.0930 - val_auc: 0.9994
Epoch 104/200
 - 9s - loss: 0.1723 - auc: 0.9966 - val_loss: 0.0929 - val_auc: 0.9995
Epoch 105/200
 - 9s - loss: 0.1874 - auc: 0.9972 - val_loss: 0.0950 - val_auc: 0.9994
Epoch 106/200
 - 9s - loss: 0.1471 - auc: 0.9986 - val_loss: 0.0881 - val_auc: 0.9995
Epoch 107/200
 - 9s - loss: 0.1746 - auc: 0.9983 - val_loss: 0.0995 - val_auc: 0.9994
Epoch 108/200
 - 9s - loss: 0.1695 - auc: 0.9971 - val_loss: 0.1137 - val_auc: 0.9994
Epoch 109/200
 - 9s - loss: 0.1733 - auc: 0.9970 - val_loss: 0.0757 - val_auc: 0.9997
Epoch 110/200
 - 9s - loss: 0.1811 - auc: 0.9978 - val_

Epoch 195/200
 - 9s - loss: 0.1644 - auc: 0.9971 - val_loss: 0.0877 - val_auc: 0.9995
Epoch 196/200
 - 9s - loss: 0.1505 - auc: 0.9972 - val_loss: 0.0844 - val_auc: 0.9997
Epoch 197/200
 - 9s - loss: 0.1441 - auc: 0.9972 - val_loss: 0.0838 - val_auc: 0.9996
Epoch 198/200
 - 9s - loss: 0.1297 - auc: 0.9976 - val_loss: 0.0901 - val_auc: 0.9996
Epoch 199/200
 - 9s - loss: 0.1596 - auc: 0.9976 - val_loss: 0.0786 - val_auc: 0.9997
Epoch 200/200
 - 9s - loss: 0.1591 - auc: 0.9976 - val_loss: 0.0863 - val_auc: 0.9997
SMOTETomek,MIT-BIH Normal Sinus Rhythm Data
finish
